In [ ]:
import pandas as pd
from pp_netlib.network import Network
datafile = "strep_MLST.tsv"
data = pd.read_csv(datafile, sep= "\t", header=0)

## cleaning up the data a little
data.drop("clonal_complex", axis = 1, inplace=True)
data["ST"] = "ST_" + data["ST"].astype(str)
data.index = data["ST"]
data.drop("ST", axis = 1, inplace=True)
data

In [ ]:
data["mlst"] = data[["aroE", "gdh", "gki", "recP", "spi", "xpt", "ddl"]].values.tolist()
data_dict = dict(zip(data.index, data["mlst"]))
data_dict

In [ ]:
def hamming_distance(list1, list2):
    match_score = 0
    for i in range(len(list1)):
        if list1[i] ==  list2[i]:
            match_score += 1

    return match_score


In [ ]:
mlst_dist_df = pd.DataFrame(index = data.index, columns = data.index)
sources = []
targets = []
weights = []
for i in range(len(data.index)):
    for j in range(i+1, len(data.index)):
        ham_dist = hamming_distance(data_dict[data.index[i]], data_dict[data.index[j]])
        mlst_dist_df.loc[data.index[i], data.index[j]] = ham_dist
        mlst_dist_df.loc[data.index[j], data.index[i]] = ham_dist
        sources.append(data.index[i])
        targets.append(data.index[j])
        weights.append(ham_dist)

mlst_dist_df

In [ ]:
mlst_dist_df.fillna(7, inplace=True)
mlst_dist_df

In [ ]:
mlst_dist_df.to_csv("mlst_distances.csv", sep = ",", header = True, index = True)

In [ ]:
edge_list = []
for i in range(len(mlst_dist_df.index)):
    for j in range(i+1, len(mlst_dist_df.index)):
        if mlst_dist_df.loc[mlst_dist_df.index[i], mlst_dist_df.index[j]] > 6:
            edge_list.append((mlst_dist_df.index[i], mlst_dist_df.index[j], mlst_dist_df.loc[mlst_dist_df.index[i], mlst_dist_df.index[j]]))

print(edge_list[:5])
print(len(edge_list))

In [ ]:
all_edges = list(zip(sources, targets, weights))
weights_gt_5 = []
for i in range(len(weights)):
    if weights[i] > 5:
        weights_gt_5.append((sources[i], targets[i], weights[i]))
        
print(len(all_edges))
print(len(weights_gt_5))

In [ ]:
from pp_netlib.network import Network

mlst_network = Network(ref_list=data.index, backend="GT")
mlst_network.construct(network_data=weights_gt_5, weights=True)

In [ ]:
filt_sources, filt_targets, filt_weights = zip(*weights_gt_5)

In [ ]:
len(filt_sources)

In [ ]:
filt_df = pd.DataFrame()
filt_df["source"] = filt_sources
filt_df["target"] = filt_targets
filt_df["weight"] = filt_weights

filt_df

In [ ]:
filt_df.to_csv("mlst_dist_gt5.csv", sep = "\t", header = True, index = False)

In [ ]:
all_df = pd.DataFrame()
all_df["source"] = sources
all_df["target"] = targets
all_df["weight"] = weights

all_df.to_csv("mlst_all_dists.csv", sep = "\t", header = True, index = False)